Base de datos Spotify.

In [ ]:
# Importar spotity
!pip install spotipy
import spotipy

In [ ]:
# Para entrar:
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# Credenciales
CLIENT_ID = '7a9658b7430848dcbe6bdf5465b1afde'
CLIENT_SECRET = 'c80b14b55a09441ebeaf685e5113deff'

#Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)
redirect_uri="http://localhost:8888/callback",
scope="user-library-read"  # Ajusta los permisos según necesites

In [ ]:
# Géneros que usaremos: Rock, Pop, Latino y R&B.
# Tramo: 2020 -2025
genres = ['rock', 'pop', 'latino' , 'R&B']
year_start = 2020
year_end = 2025
limit= 50
results = []
artists = []

#Aquí iríamos buscando por género
datos = sp.search(q=f"genre: rock year:{year_start}-{year_end}", type='track,album', limit= 50, offset=0)

In [ ]:
for genre in genres:
    for offset in range(0, 1000, limit):  # Ajusta el rango según la cantidad de resultados que quieras
        results_genre = sp.search(q=f'genre:{genre} year:{year_start}-{year_end}', type='track,album', limit=limit, offset=offset)
        for item in results_genre ['tracks']['items']: # Entra a cada canción
            artists.append(item['artists'][0]['name']) # Añade el nombre del artista de la canción
            results.append({
                'name_track': item['name'],
                'name_artist': item['artists'][0]['name'],
                'genre': genre, # Hacer que vaya pasando por cada género solo
                'type': item['type'],
                'year': item['album']['release_date'][:4] }) # Para que solo recoja el año ustamos :4, debido al formato 01/12/2024
        for item in results_genre['albums']['items']: # Entra a cada album
            artists.append(item['artists'][0]['name']) # Añade el nombre del artista del album
            results.append({
                'name_track': item['name'],
                'name_artist': item['artists'][0]['name'],
                'genre': genre, # Hacer que vaya pasando por cada género solo
                'type': item['type'], 
                'year': item['release_date'][:4] })# Para que solo recoja el año ustamos :4, debido al formato 01/12/2024
len(results)

In [ ]:
print(results)


In [ ]:
import pandas as pd
df_spotify= pd.DataFrame(results)

In [ ]:
df_spotify

In [ ]:
df_spotify.to_csv('spotify_data_tracks.csv', index=False, encoding='utf-8')

Base de datos last.fm.


In [ ]:
# Importar librería para la conexión con MySQL
import mysql.connector
from mysql.connector import errorcode

import requests
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:

# API Key obtenida de last.fm
API_KEY =  'be844a0247f4741ca1485d075e544319' # Reemplaza con tu API Key válida de Last.fm.
BASE_URL = 'http://ws.audioscrobbler.com/2.0/' # esta base siempre es la misma.

In [ ]:
import requests
import time
import pandas as pd

# Lista para almacenar los datos que luego guardaremos en un DataFrame
artist_data = []

# Elimina duplicados si es necesario
artists = list(set(artists))

for artist_name in artists:
    params_info = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': API_KEY,
        'format': 'json',
        'lang': 'es'  # Intentar obtener la biografía en español
    }
    try:
        response_info = requests.get(BASE_URL, params=params_info)
        data_info = response_info.json()

        if 'artist' in data_info:
            artist_info = data_info['artist']
            name = artist_info['name']
            listeners = artist_info['stats']['listeners']
            playcount = artist_info['stats']['playcount']
            biography = artist_info['bio']['content']
            similar_artists = [a['name'] for a in artist_info['similar']['artist']]

            artist_data.append({
                'artista': name,
                'oyentes': listeners,
                'reproducciones': playcount,
                'biografia': biography,
                'artistas_similares': similar_artists
            })
        else:
            print(f"No se encontró información para el artista: {artist_name}")
            # Puedes decidir si agregar una entrada con valores nulos o simplemente continuar
            artist_data.append({
                'artista': artist_name,
                'oyentes': None,
                'reproducciones': None,
                'biografia': None,
                'artistas_similares': None
            })
    except Exception as e:
        print(f"Error al obtener datos del artista '{artist_name}': {e}")
        # Manejo de excepciones; puedes optar por registrar el error y continuar
        artist_data.append({
            'artista': artist_name,
            'oyentes': None,
            'reproducciones': None,
            'biografia': None,
            'artistas_similares': None
        })
    # Pausa para respetar los límites de la API
    time.sleep(0.2)  # Ajusta el tiempo según las políticas actuales de Last.fm


In [ ]:
df_artistas = pd.DataFrame(artist_data)


In [ ]:
df_artistas

In [ ]:
df_artistas.to_csv('spotify_lastfm_data.csv', index=False, encoding='utf-8')


Exportar las bases de datos a SQL.

In [ ]:
!pip install sqlalchemy
!pip install mysql-connector-python

In [ ]:
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd
# Conectar a MySQL
conn = mysql.connector.connect(user='root', password='AlumnaAdalab', host='localhost')
cursor = conn.cursor()
# Crear el esquema 'nuevo_schema' si no existe
cursor.execute("CREATE DATABASE IF NOT EXISTS music_stream")
print("Esquema 'nuevo_schema' creado correctamente.")


In [ ]:

user = 'root'
password = 'AlumnaAdalab'
host = 'localhost'
database = 'music_stream'
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')
# Exportar el DataFrame a MySQL
df_spotify.to_sql('spotify', con=engine, if_exists='replace', index=False)
print("Datos exportados correctamente a MySQL.")
# Exportar el DataFrame a MySQL
df_artistas.to_sql('last_fm', con=engine, if_exists='replace', index=False)
print("Datos exportados correctamente a MySQL.")

Manejamos un error, en el que los artitas similares no se nos había pasado como cadena de texto.

In [ ]:

df = df_artistas

# Convertir las listas en la columna 'artistas_similares' a cadenas de texto
df['artistas_similares'] = df['artistas_similares'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

user = 'root'
password = 'AlumnaAdalab'
host = 'localhost'
database = 'music_stream'

# Definir la cadena de conexión (ajusta según tu configuración)
connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'

# Crear el motor de SQLAlchemy
engine = create_engine(connection_string)

# Insertar los datos en la base de datos
try:
    df.to_sql('last_fm', con=engine, if_exists='append', index=False)
    print("Datos insertados exitosamente")
except Exception as e:
    print(f"Ocurrió un error: {e}")


Cerrar conexión y cursor.

In [ ]:
cursor.close()
conn.close()